# Chapter 8 recap Training a model for movie review classification

In [ ]:
import nltk
nltk.download("stopwords")

In [2]:
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

LookupError: 
**********************************************************************
  Resource 'corpora/stopwords' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - '/root/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

In [ ]:
next(stream_docs(path="./movie_data.csv"))

In [ ]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [6]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error="ignore",
                         n_features=2**21,
                         preprocessir=None,
                         tokenizer=tokenizer)

clf = SGDClassifier(loss="log", random_state=1, n_iter=1)
doc_stream = stream_docs(path='./data/movie_data.csv')

NameError: name 'tokenizer' is not defined

In [ ]:
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

In [ ]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print("Accuracy: %.3f" % clf.score(X_test, y_test))
clf = clf.partial_fit(X_test, y_test)

In [ ]:
import sqlite3
import os
conn = sqlite3.connect("review.sqlite")
c = conn.cursor()
c.execute("CREATE TABLE review_db (review TEXT, sentiment INTERGER, date TEXT)")
example1 = "I love this movie"
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES" "(?, ?, DATETIME('now'))", (example1, 1))
example2 = "I disliked this movie"
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES" "(?, ?, DATETIME('now'))", (example2, 0))
conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect("reviews.sqlite")
c = conn.cursor()
c.execute("SELECT * FROM review_db WHERE date BETWEEN '2016-01-01 00:00:00' AND DATETIME('now')")
results = c.fetchall()
conn.close()
print(results)